In [2]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 MB 81.2 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 74.3 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.2
    Uninstalling sympy-1.13.2:
      Successfully uninstalled sympy-1.13.2


In [3]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
user_question = "I just discovered the course. Can I still join it?"
v = embedding_model.encode(user_question)


In [7]:
len(v)

768

In [65]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models
qd_client = QdrantClient("http://localhost:6333")

In [11]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [13]:
collection_name = "zoomcamp-faq"
qd_client.delete_collection(collection_name=collection_name)

False

In [20]:
query = 'I just discovered the course. Can I join now?'
vector = models.Document(text=text, model=model_handle)
# min(vector)

In [21]:
embedding_model = TextEmbedding(model_name=model_handle)

In [22]:
query_embedding = list(embedding_model.embed([query]))[0]

In [24]:
min(query_embedding)

-0.11726373551188797

In [25]:
query_embedding.dot(query_embedding)

1.0000000000000004

In [26]:
doc = 'Can I still join the course after the start date?'
doc_embedding = list(embedding_model.embed([doc]))[0]

In [27]:
query_embedding.dot(doc_embedding)

0.9008528856818033

In [14]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [15]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword"
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

In [28]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [29]:
query = 'Can I still join the course after the start date?'
query_embedding = list(embedding_model.embed([query]))[0]

"Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute. Course - Can I still join the course after the start date?"

In [40]:
import numpy as np

In [50]:
doc_vectors = []
for i, doc in enumerate(documents):
    # text = 'answer: ' + doc['text'] + ' question: ' + doc['question'] 
    text =  doc['text'] 
    doc_embedding = list(embedding_model.embed([text]))[0]
    # print(len(doc_embedding))
    doc_vectors.append(doc_embedding)
    doc_vectors_ar = np.array(doc_vectors)
    # doc_vectors = np.array(doc_vectors)
    
doc_vectors_ar.dot(query_embedding)


array([0.8106057 , 0.84989067, 0.7991335 , 0.68130469, 0.6739288 ])

In [51]:
doc_vectors = []
for i, doc in enumerate(documents):
    # text = 'answer: ' + doc['text'] + ' question: ' + doc['question'] 
    text =  doc['question'] + ' ' + doc['text']
    doc_embedding = list(embedding_model.embed([text]))[0]
    # print(len(doc_embedding))
    doc_vectors.append(doc_embedding)
    doc_vectors_ar = np.array(doc_vectors)
    # doc_vectors = np.array(doc_vectors)
    
doc_vectors_ar.dot(query_embedding)


array([0.92732519, 0.85948473, 0.83966023, 0.76320296, 0.76044492])

array([0.8106057 , 0.84989067, 0.7991335 , 0.68130469, 0.6739288 ])

In [37]:
len(doc_vectors)
for i, v in enumerate(doc_vectors):
    print(i, v.dot(query_embedding))

0 0.8953807324518879
1 0.8523042316413953
2 0.8129676765715639
3 0.7470864177415646
4 0.7346513026969572


In [52]:
from fastembed import TextEmbedding
models = TextEmbedding.list_supported_models()

In [57]:
# for model in models:
#     if model['model'] == 'BAAI/bge-small-en':


model_fil = [model for model in models if model['model'] == 'BAAI/bge-small-en']

In [59]:
smallest_model_by_dim = min(model_fil, key=lambda model: model['dim'])

In [60]:
smallest_model_by_dim

{'model': 'BAAI/bge-small-en',
 'sources': {'hf': 'Qdrant/bge-small-en',
  'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz',
  '_deprecated_tar_struct': True},
 'model_file': 'model_optimized.onnx',
 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
 'license': 'mit',
 'size_in_GB': 0.13,
 'additional_files': [],
 'dim': 384,
 'tasks': {}}

In [78]:
EMBEDDING_DIMENSIONALITY = 384
model_handle = 'BAAI/bge-small-en'

In [79]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models
qd_client = QdrantClient("http://localhost:6333")
collection_name = "zoomcamp-faq"
qd_client.delete_collection(collection_name)
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [80]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [81]:
points = []
id = 0

# for course in documents:
for doc in documents:
    text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=id,
        vector=models.Document(text=text, model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": doc['course']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

In [82]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [83]:
q1 = 'I just discovered the course. Can I join now?'

In [86]:
def search(query, limit=1):

    results = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [87]:
search(q1)

QueryResponse(points=[ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])